In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
%run constants.ipynb
%run functions.ipynb

In [3]:
tabs = ['default','bars']
cate = ['Seat', 'Food']
manu = ['Airbus', 'Boeing', 'Cessna', 'Cirrus ','Pilatus', 'Matra']
airl = list(df_airline['airline'].apply(preprocess))
airc = list(df_aircraft['model'].apply(preprocess))

##### make the database

In [4]:
bdd = make_bdd(50)
bdd.head()

,session,date,json
0,0,09/06/2015 14h21m09s,"{""session"": 0, ""filters"": {""manufacturer"": [""C..."
1,0,25/00/2014 00h42m08s,"{""session"": 0, ""filters"": {""manufacturer"": [],..."
2,0,05/04/2013 13h45m20s,"{""session"": 0, ""filters"": {""manufacturer"": [""C..."
3,0,08/06/2019 07h39m41s,"{""session"": 0, ""filters"": {""manufacturer"": [""B..."
4,0,01/00/2015 23h17m58s,"{""session"": 0, ""filters"": {""manufacturer"": [""C..."


##### hash the jsons : create a single representation for every corresponding json

In [5]:
bdd[CT_bdd_hash] = bdd[CT_bdd_json].apply(json_string_to_hash)

##### extract the states

In [6]:
etats = dict([(a,b) for (b,a) in enumerate(sorted(bdd[CT_bdd_hash].unique()))])
id_to_etats = dict((b,a) for (a,b) in etats.items())

##### extract the transitions

In [7]:
transitions = np.zeros([len(etats),len(etats)])
for i in range(1,len(bdd)):
    transitions[etats[bdd[CT_bdd_hash][i-1]], etats[bdd[CT_bdd_hash][i]]] += 1

# to proba matrix
transitions = (transitions.T / np.sum(transitions,axis=1)).T
df_transitions = pd.DataFrame(transitions, columns=etats, index=etats)

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


##### make a prediction

In [8]:
a = id_to_etats[0]

predict_next_state(a, df_transitions)

'{"filters": {"manufacturer": ["cirrus ", "matra"], "aircraft": ["c_101_aviojet", "c_212_aviocar", "mig_33", "su_26"], "company": ["kalitta_air"], "category": ["food"], "date": ["08012010", "08102019"]}, "tab": "default"}'